In [ ]:
import sys
import time
import datetime
from train import *
from params import TrainingParams

def time_remaining(params, batch):
    time_remaining = 0
    total_batches = (params.total_train_sample/params.batch_size)*params.num_epoch
    
    if params.initial_batch is None:
        params.initial_batch = batch
        params.last_time = time.perf_counter()
    else:
        cur_time = time.perf_counter()
        elapsed = cur_time - params.last_time
        params.last_time = cur_time
        batch_remaining = total_batches - (batch-params.initial_batch)
        time_remaining = int(batch_remaining*elapsed)
    return time_remaining, (total_batches+params.initial_batch-1)

def report_loss_simple(params, batch, total_cost, content_cost, style_cost):
    t, tb = time_remaining(params, batch)
    sys.stdout.write("\r")
    sys.stdout.write("Batch %i/%i. Total Loss: %s. ETA %s         " % (batch, tb, str(total_cost), str(datetime.timedelta(seconds=t))))
    sys.stdout.flush()

In [ ]:
params = TrainingParams()
# aws
# params.train_path='/home/ubuntu/work/data/unlabeled2017/*.jpg'
# mbp
# params.train_path='/Users/paul/Work/ai/images/val2017/*.jpg'
# azure
# params.train_path = '/home/paul/src/images/train2017/*.jpg'
# tf
params.train_path = ['/Users/paul/Work/ai/images/tf/%i.tfr' % (i) for i in range(2)]
params.style_path='data/mosaic.jpg'
params.batch_size = 4
params.num_epoch = 1
params.learn_rate = 0.0001
params.total_train_sample = 40
params.style_weight = 5.0
params.content_weight = 1.0
params.tv_weight=0.0
params.save_path='summaries'
params.summary_step=5

In [ ]:
params.last_time = None
params.initial_batch = None
train(params, report_fn=report_loss_simple, start_new=True)

In [ ]:
# needed for style an image
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
from params import TransferParams
from pipeline import *
import numpy as np

def transfer(path, out_path=None):
    # init    
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    
    input_image = process_img(path, crop=False).eval()
    input_shape = [1] + list(input_image.shape)
    
    # make the model
    input_placeholder = tf.placeholder(dtype=tf.float32, shape=input_shape, name='input_images')
    gen = SpriteGenerator(input_placeholder, 'SpriteGenerator')
    
    # restore
    saver = tf.train.Saver()
    saver.restore(sess, 'data/model/style.ckpt')
    
    # generate and write out
    output = sess.run(gen.output, feed_dict={input_placeholder:np.stack([input_image])})
    plt.imshow(output[0]/255.)  
    
    if out_path is not None:
        output = tf.image.encode_jpeg(output[0])  
        write = tf.write_file(out_path, output)
        sess.run(write)
        print('Generate Done.')

In [ ]:
# transfer('data/test.jpg', 'data/generated.jpg')